Data set: Online Retail II data set contains the sales records of an online retail store based in the UK between 01/12/2009 and 09/12/2011.

Business Problem: Making product recommendations to users at the cart stage
Note: If 'InvoiceNo' starts with 'C', transaction is cancelled.

In [1]:
!pip install mlxtend
import pandas as pd
from numpy.ma.core import product
from mlxtend.frequent_patterns import apriori, association_rules

pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.width', 500)
pd.set_option('display.expand_frame_repr', False)

**DATA PREPARATION**

In [2]:
df = pd.read_excel("/kaggle/input/online-retails/online_retail_II.xlsx",
                    sheet_name="Year 2010-2011")
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [3]:
df.describe().T

# Quantity and Price variables have some missing values
# Cancelled Invoice values causes outlier values

,count,mean,min,25%,50%,75%,max,std
Quantity,541910.0,9.552234,-80995.0,1.0,3.0,10.0,80995.0,218.080957
InvoiceDate,541910,2011-07-04 13:35:22.342307584,2010-12-01 08:26:00,2011-03-28 11:34:00,2011-07-19 17:17:00,2011-10-19 11:27:00,2011-12-09 12:50:00,NaN
Price,541910.0,4.611138,-11062.06,1.25,2.08,4.13,38970.0,96.759765
Customer ID,406830.0,15287.68416,12346.0,13953.0,15152.0,16791.0,18287.0,1713.603074


In [4]:
# Function to calculate outlier thresholds:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquartile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquartile_range
    low_limit = quartile1 - 1.5 * interquartile_range
    return low_limit, up_limit

# Function to cap outliers at threshold values:
def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

In [5]:
df.dropna(inplace=True)
df = df[~df["Invoice"].str.contains("C", na=False)]
df = df[df["Quantity"] > 0]
df = df[df["Price"] > 0]

replace_with_thresholds(df, "Quantity")
replace_with_thresholds(df, "Price")
df.describe().T

/tmp/ipykernel_17/3290064968.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-177.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit


,count,mean,min,25%,50%,75%,max,std
Quantity,397885.0,11.83077,1.0,2.0,6.0,12.0,298.5,25.523052
InvoiceDate,397885,2011-07-10 23:41:56.419316992,2010-12-01 08:26:00,2011-04-07 11:12:00,2011-07-31 14:39:00,2011-10-20 14:33:00,2011-12-09 12:50:00,NaN
Price,397885.0,2.893492,0.001,1.25,1.95,3.75,37.06,3.227175
Customer ID,397885.0,15294.416882,12346.0,13969.0,15159.0,16795.0,18287.0,1713.144421


**Invoice-Product Matrix and Pivot Operation**

In [6]:
# Filter the dataset to only include transactions from "France"
df_fr = df[df["Country"] == "France"]

# Grouping by Invoice and Description to get the total quantity of each product in each invoice
df_fr.groupby(['Invoice', 'StockCode']).agg({"Quantity": "sum"}).head(20)

# Pivoting the table: Invoices as rows, products as columns
pivot = df_fr.pivot_table(index="Invoice", columns="StockCode", values="Quantity", aggfunc="sum").notnull().astype(int)
pivot.head()

StockCode,10002,10120,10125,10135,11001,15036,15039,16012,16048,16218,16219,16225,16236,16237,16238,17174,20615,20617,20658,20665,20668,20674,20675,20676,20677,20679,20681,20682,20684,20685,20686,20702,20704,20711,20712,20713,20717,20718,20719,20723,20724,20725,20726,20727,20728,20749,20750,20751,20767,20777,20819,20832,20866,20914,20961,20963,20966,20967,20971,20972,20973,20974,20975,20977,20978,20979,20981,20983,20984,20992,20996,21012,21026,21027,21030,21034,21035,21039,21042,21054,21055,21056,21058,21059,21061,21062,21063,21064,21065,21068,21069,21070,21078,21080,21084,21086,21087,21088,21090,21094,21095,21096,21098,21108,21110,21111,21114,21116,21121,21122,21123,21124,21125,21126,21135,21136,21137,21154,21155,21156,21161,21162,21163,21164,21165,21166,21174,21175,21181,21188,21194,21195,21196,21197,21198,21199,21200,21201,21202,21204,21205,21206,21207,21208,21209,21210,21212,21213,21216,21217,21218,21219,21221,21224,21231,21232,21238,21239,21240,21242,21243,21244,21245,21246,21248,21249,21251,21257,21258,21259,21262,21284,21286,21294,21306,21326,21328,21329,21340,21351,21354,21355,21356,21358,21361,21365,21369,21371,21372,21374,21375,21377,21380,21381,21383,21385,21394,21395,21398,21399,21402,21403,21407,21408,21411,21424,21425,21426,21427,21428,21429,21430,21439,21442,21452,21463,21466,21467,21469,21470,21471,21472,21479,21481,21484,21485,21494,21495,21497,21498,21499,21500,21506,21507,21508,21509,21519,21523,21524,21527,21528,21531,21533,21535,21537,21539,21544,21555,21556,21557,21558,21559,21561,21563,21564,21576,21577,21578,21579,21581,21584,21591,21621,21622,21623,21624,21643,21648,21650,21658,21668,21669,21671,21672,21673,21675,21677,21678,21679,21680,21693,21696,21698,21700,21703,21704,21706,21707,21708,21709,21710,21711,21715,21716,21719,21721,21723,21724,21725,21731,21733,21739,21746,21747,21749,21754,21755,21756,21770,21774,21786,21787,21790,21791,21812,21813,21818,21819,21821,21828,21829,21832,21833,21843,21844,21845,21864,21865,21866,21871,21872,21875,21877,21880,21881,21882,21883,21884,21888,21889,21890,21891,21892,21894,21900,21901,21905,21906,21907,21908,21912,21913,21914,21915,21916,21917,21918,21922,21926,21928,21929,21930,21931,21932,21933,21934,21935,21936,21937,21943,21946,21947,21948,21949,21955,21967,21972,21974,21975,21976,21977,21980,21981,21982,21983,21985,21986,21987,21988,21989,21990,22023,22024,22025,22026,22027,22028,22029,22032,22035,22037,22041,22043,22044,22045,22046,22051,22052,22059,22063,22064,22065,22068,22070,22071,22072,22073,22076,22077,22078,22079,22080,22081,22082,22083,22084,22085,22086,22087,22088,22089,22090,22091,22093,22094,22099,22109,22110,22111,22112,22113,22114,22115,22116,22118,22120,22121,22124,22129,22130,22131,22132,22133,22134,22136,22138,22139,22141,22142,22144,22149,22150,22151,22153,22170,22174,22175,22176,22178,22179,22180,22181,22189,22191,22192,22193,22195,22197,22198,22199,22200,22203,22204,22207,22208,22209,22210,22211,22215,22219,22221,22222,22223,22224,22228,22231,22232,22236,22242,22243,22244,22254,22255,22262,22264,22269,22271,22272,22273,22274,22282,22285,22294,22295,22299,22300,22301,22302,22303,22305,22306,22309,22311,22312,22313,22314,22315,22318,22319,22320,22321,22322,22324,22325,22326,22327,22328,22329,22331,22332,22333,22334,22339,22340,22341,22343,22344,22346,22348,22349,22350,22352,22354,22355,22356,22357,22358,22361,22362,22365,22366,22367,22371,22372,22374,22375,22376,22377,22378,22379,22380,22381,22382,22383,22384,22385,22386,22389,22390,22393,22394,22396,22398,22399,22400,22402,22403,22406,22408,22411,22412,22413,22414,22416,22417,22418,22419,22422,22423,22425,22427,22431,22432,22433,22434,22435,22436,22437,22438,22439,22440,22443,22444,22445,22449,22451,22452,22453,22454,22456,22457,22459,22466,22467,22468,22469,22471,22472,22474,22475,22480,22482,22483,22485,22489,22491,22492,22493,22495,22498,22499,22500,22501,22502,22504,22505,22507,22508,22514,22515,22516,22517,22518,22519,22520,22521,22522,22523,22524,22525,22526,22529,22530,22531,22532,

**Generating Association Rules**

In [7]:

frequent_itemsets = apriori(pivot, min_support=0.01, use_colnames=True)

rules = association_rules(frequent_itemsets, metric="support", min_threshold=0.01)

rules.head()

/opt/conda/lib/python3.10/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(10002),(21791),0.020566,0.028278,0.010283,0.500000,17.681818,0.009701,1.943445,0.963255
1,(21791),(10002),0.028278,0.020566,0.010283,0.363636,17.681818,0.009701,1.539111,0.970899
2,(10002),(21915),0.020566,0.069409,0.010283,0.500000,7.203704,0.008855,1.861183,0.879265
3,(21915),(10002),0.069409,0.020566,0.010283,0.148148,7.203704,0.008855,1.149771,0.925414
4,(10002),(22551),0.020566,0.136247,0.010283,0.500000,3.669811,0.007481,1.727506,0.742782


**Function to return the product name when given a StockCode from a DataFrame**

In [8]:
def check_id(dataframe, stock_code):
    product_name = dataframe[dataframe["StockCode"] == stock_code][["Description"]].values[0].tolist()
    print(product_name)
    
check_id(df_fr, 10002)

['INFLATABLE POLITICAL GLOBE ']


**Association Rule Learning Recommender**

In [9]:
def arl_recommender(rules_df, product_id, rec=1):
    sorted_rules = rules_df.sort_values("lift", ascending=False)
    recommendation_list = []
    for i, product in enumerate(sorted_rules["antecedents"]):
        for j in list(product):
            if j == product_id :
                for k in list(sorted_rules.iloc[i]["consequents"]):
                    if k not in recommendation_list:
                        recommendation_list.append(k)

    return recommendation_list[0:rec]

In [10]:
product_id = 10002
check_id(df_fr, product_id)

['INFLATABLE POLITICAL GLOBE ']


In [11]:
arl_recommender(rules, product_id, 1)

[21791]